This notebook allowed to make a gtiff file of incidence angle in the DEM geometry.

In [ ]:
%load_ext autoreload
%autoreload 2
from eo_tools.S1.core import S1IWSwath
from eo_tools.S1.core import load_dem_coords, lla_to_ecef

from eo_tools_dev.util import show_insar_phi

from eo_tools.S1.util import presum
import numpy as np
import os
import matplotlib.pyplot as plt

import logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

# Parameter setup

In [ ]:
# change with directory containing your S1 products
data_dir = "/data/S1"
# change with directory containing the results
out_dir = "/data/res/test-rtc"
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

# replace with already downloaded and unzipped products (see the other notebooks to download such products)
primary_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230904T063730_20230904T063757_050174_0609E3_DAA1.zip"
secondary_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230916T063730_20230916T063757_050349_060FCD_6814.zip"

# subswath to process
iw = 1
# polarization
pol = "vv"
# DEM upsampling
up = 1.8

min_burst = 4
max_burst = 5

# Process individual bursts

In [ ]:
slc = []
lut = []
dems = []
sigma_ts = []

prm = S1IWSwath(primary_dir, iw=iw, pol=pol)
sec = S1IWSwath(secondary_dir, iw=iw, pol=pol)
overlap = np.round(prm.compute_burst_overlap(2)).astype(int)

for burst_idx in range(min_burst, max_burst + 1):
    # log.info(f"---- Processing burst {burst_idx} ----")

    # compute geocoding LUTs for master and slave bursts
    file_dem = prm.fetch_dem_burst(burst_idx, dir_dem="/data/tmp", force_download=False)
    az_p2g, rg_p2g, _, sigma_t = prm.geocode_burst(
        file_dem, burst_idx=burst_idx, dem_upsampling=up,
    )

    # read primary and secondary burst raster
    arr_p = prm.read_burst(burst_idx, True)
    
    dem_coords = load_dem_coords(file_dem=file_dem, upscale_factor=up)
    dem_x, dem_y, dem_z = lla_to_ecef(dem_coords[0], dem_coords[1], dem_coords[2], None)
    # arr_p *= sigma_t

    # radiometric calibration 
    cal_p = prm.calibration_factor(burst_idx, cal_type="beta")
    arr_p /= cal_p

    slc.append(np.abs(arr_p))
    sigma_ts.append(sigma_t)
    lut.append((az_p2g, rg_p2g))
    dems.append(file_dem)

# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 10))
# plt.imshow(np.log(sigma_t[:,::4].clip(1e-8)), interpolation="bilinear", vmin = -5, vmax=0)
# plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(sigma_ts[1][:,:], interpolation="none",cmap="grey")
# plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
## DBG: angle
from eo_tools.S1.core import resample
import rioxarray as riox
from rioxarray.merge import merge_arrays
from eo_tools.auxils import remove
import rasterio

out_name = f"{out_dir}/merged_proj_angle.tif"

list_ang = []
files_to_remove = []
for i in range(min_burst, max_burst + 1):
    log.info(f"Burst {i}")

    file_dem = dems[i - min_burst]
    with rasterio.open(file_dem) as ds_dem:
        prof = ds_dem.profile.copy()

    file_ang = f"{out_dir}/ang_{i}.tif"
    prof.update({"dtype": sigma_ts[i - min_burst].dtype.name, "nodata": np.nan})
    with rasterio.open(file_ang, "w", **prof) as ds_ang:
        ds_ang.write(sigma_ts[i - min_burst], 1)
    files_to_remove.append(file_ang)
    list_ang.append(riox.open_rasterio(file_ang))

merged_ang = merge_arrays(list_ang)
# merged_ifg = merge_arrays(list_ifg, nodata=0, method="last")
merged_ang.rio.to_raster(out_name)
# merged_ifg.rio.to_raster(f"{out_dir}/merged_img0.tif")
print(files_to_remove)
for fname in files_to_remove:
    remove(fname)

In [ ]:
ds1 = riox.open_rasterio(f"{out_dir}/merged_proj_angle.tif")
ds2 = riox.open_rasterio(f"{out_dir}/merged_angle.tif")

(ds1[0]-ds2[0]).plot.imshow()